**RECOMMENDATION SYSTEM:**

*Task to Implement:*

Create a simple recommendation system that suggests items to users
based on their preferences. You can use techniques like collaborative
filtering or content-based filtering to recommend movies, books, or
products to users.



**Step 1:** Prepare the Data

We will create a larger dataset of user-item interactions with random ratings.

In [6]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

#  To Generate a larger dataset:
np.random.seed(42)
users = [f'User{i}' for i in range(1, 21)]  # 20 users
items = [f'Movie{i}' for i in range(1, 21)] # 20 items
data = []

for user in users:
    for item in items:
        if np.random.rand() > 0.7:  # 30% chance that the user has rated the movie
            rating = np.random.randint(1, 6)  # Rating between 1 and 5
            data.append([user, item, rating])

# Covert data to Data Frame
df = pd.DataFrame(data, columns=['User', 'Item', 'Rating'])
print(df.head())


    User     Item  Rating
0  User1   Movie2       3
1  User1   Movie3       5
2  User1   Movie7       4
3  User1  Movie11       2
4  User2   Movie3       3


**Step 2**: Compute User Similarity

We will compute the user similarity matrix using *cosine similarity*.

In [5]:
# Create a user-item matrix
user_item_matrix = df.pivot_table(index='User', columns='Item', values='Rating').fillna(0)
user_item_matrix_sparse = csr_matrix(user_item_matrix.values)

# Compute cosine similarity between users
user_similarity = cosine_similarity(user_item_matrix_sparse)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

print(user_similarity_df.head())


User       User1    User10    User11    User12    User13    User14    User15  \
User                                                                           
User1   1.000000  0.000000  0.228218  0.587945  0.296586  0.254588  0.000000   
User10  0.000000  1.000000  0.273861  0.083992  0.124878  0.000000  0.547723   
User11  0.228218  0.273861  1.000000  0.224271  0.525814  0.149404  0.450000   
User12  0.587945  0.083992  0.224271  1.000000  0.017700  0.206197  0.184017   
User13  0.296586  0.124878  0.525814  0.017700  1.000000  0.367884  0.000000   

User      User16    User17    User18    User19     User2    User20     User3  \
User                                                                           
User1   0.379049  0.365434  0.000000  0.406625  0.285831  0.000000  0.254837   
User10  0.000000  0.536731  0.348651  0.048795  0.285831  0.503506  0.382255   
User11  0.166091  0.469117  0.286446  0.293987  0.093934  0.413673  0.340226   
User12  0.429801  0.051795  0.098833  0

**Step 3:** Generate Recommendations

We will generate recommendations for a user based on similar users preferences

In [7]:
def get_recommendations(user, user_similarity_df, user_item_matrix, n_recommendations=5):
    #Get the series of similar users, sorted by similarity
    similar_users = user_similarity_df[user].sort_values(ascending=False).index[1:]

    # Get the ratings given by the target user
    user_ratings = user_item_matrix.loc[user]

    # Initialize a dictionary to store the recommendations
    recommendations = {}

    # Iterate through similar users
    for similar_user in similar_users:
        similar_user_ratings = user_item_matrix.loc[similar_user]

        # Iterate through the items rated by the similar user
        for item in similar_user_ratings.index:
            if user_ratings[item] == 0 and similar_user_ratings[item] > 0:
              # Recommend items not rated by the user
                if item not in recommendations:
                    recommendations[item] = similar_user_ratings[item]
                else:
                    recommendations[item] += similar_user_ratings[item]

    # Sort recommendations by rating and return the top N recommendations
    recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)
    return [item for item, rating in recommendations[:n_recommendations]]



Recommendations for User1: ['Movie12', 'Movie4', 'Movie16', 'Movie5', 'Movie18']


**Step 4**: Get the Recommendation from the user.



In [9]:
# Get recommendations for a specific user
user = 'User1'
recommendations = get_recommendations(user, user_similarity_df, user_item_matrix)
print(f"Recommendations for {user}: {recommendations}")

Recommendations for User1: ['Movie12', 'Movie4', 'Movie16', 'Movie5', 'Movie18']
